In [1]:
import pickle
import pandas as pd 
from nltk.tokenize import RegexpTokenizer
import gensim
from gensim.models import Word2Vec
from scipy.spatial.distance import cosine
import numpy as np

1. Load the data using Pickle (pickle.load(open(’data.p’, ’rb’)). Op- posed to the previous exercises, the data is not in a dataframe, but it is a list of sentences.

In [2]:
gender = pickle.load(open('/Users/jaspervogelzang/Documents/ADS Master/Data Mining/Lab Sessions/gender.p', 'rb'))

In [3]:
gender = [x.lower() for x in gender]
gender[0:5]

['finally, some women journalists mentioned that information and communication technologies are responsible for creating new barriers for women in journalism because of the increased pace and pressure on their private lives.',
 'still, it is necessary to first establish that language itself might play a bias-inducing role before assessing whether such bias can be overcome via another mechanism.',
 'the process of constructing a national identity directly engages the construction of gender (charrad 2001; kandiyoti 1991; kim, puri, and kim-puri 2005; yuval-davis and anthias 1989), and sudan is no exception (hale 1996; nageeb 2004; tønnessen 2007).',
 'what is the point of all this that these people do [pointing at his colleagues in the shop]—knives and women and who knows what else?',
 'the first part of this article will question whether restorative justice mechanisms can be more conducive to the inclusion of women’s experiences.']

2. Tokenize the lowercased texts using NLTK (or Spacy – if you prefer), make sure only punctuations are removed, and train a word embeddings model (with an embedding size of 300 and a min count of 5) on the data.

In [6]:
tokenizer = RegexpTokenizer('\w+')
tokenized_texts = [tokenizer.tokenize(text) for text in gender]
print(tokenized_texts[0:5])

[['finally', 'some', 'women', 'journalists', 'mentioned', 'that', 'information', 'and', 'communication', 'technologies', 'are', 'responsible', 'for', 'creating', 'new', 'barriers', 'for', 'women', 'in', 'journalism', 'because', 'of', 'the', 'increased', 'pace', 'and', 'pressure', 'on', 'their', 'private', 'lives'], ['still', 'it', 'is', 'necessary', 'to', 'first', 'establish', 'that', 'language', 'itself', 'might', 'play', 'a', 'bias', 'inducing', 'role', 'before', 'assessing', 'whether', 'such', 'bias', 'can', 'be', 'overcome', 'via', 'another', 'mechanism'], ['the', 'process', 'of', 'constructing', 'a', 'national', 'identity', 'directly', 'engages', 'the', 'construction', 'of', 'gender', 'charrad', '2001', 'kandiyoti', '1991', 'kim', 'puri', 'and', 'kim', 'puri', '2005', 'yuval', 'davis', 'and', 'anthias', '1989', 'and', 'sudan', 'is', 'no', 'exception', 'hale', '1996', 'nageeb', '2004', 'tønnessen', '2007'], ['what', 'is', 'the', 'point', 'of', 'all', 'this', 'that', 'these', 'peopl

In [8]:
SIZE = 300 # dimensions of the embeddings
SG = 1 # whether to use skip-gram or CBOW (we use skip-gram)
WINDOW = 10 # the window size
N_WORKERS = 1 # number of workers to use
MIN_COUNT = 5

model = Word2Vec(size=SIZE,
                sg=SG,
                window=WINDOW, 
                min_count=MIN_COUNT,
                workers=N_WORKERS)

model.build_vocab(tokenized_texts)

model.train(tokenized_texts,
           total_examples=model.corpus_count,
           epochs=model.epochs) # grab some coffee while training

(8060130, 10645010)

3. Use the most_similar() function for some random words to test the model. Does the model produce sensible results?

In [29]:
print(model.wv.most_similar('colleagues'))
print('\n',model.wv.most_similar('shop'))
print('\n',model.wv.most_similar('identity'))

[('peers', 0.6688945293426514), ('supervisors', 0.6020901203155518), ('mentors', 0.6013844609260559), ('coworkers', 0.5986536145210266), ('ostracized', 0.5880432724952698), ('teachers', 0.5838507413864136), ('supervisor', 0.583335280418396), ('counselors', 0.5827285051345825), ('talented', 0.580131471157074), ('raters', 0.5754419565200806)]

 [('store', 0.8822088241577148), ('shops', 0.8123973608016968), ('elegant', 0.8078662753105164), ('bride', 0.805104672908783), ('owners', 0.8045867681503296), ('nights', 0.799264669418335), ('owner', 0.7969189882278442), ('flight', 0.7919496893882751), ('dresses', 0.7885294556617737), ('brides', 0.786113977432251)]

 [('identities', 0.6562486886978149), ('malleability', 0.6120238900184631), ('identifications', 0.6060490012168884), ('categorization', 0.5976635217666626), ('norwegianness', 0.5945264101028442), ('identification', 0.5914136171340942), ('fluidity', 0.5873438715934753), ('neurodivergent', 0.5601075291633606), ('politic', 0.55437362194061

4. Calculate the similarity between man and king, and between woman and king. 

In [25]:
print(model.similarity('man', 'king'))
print(model.similarity('woman', 'king'))

0.385419
0.22175859


<ipython-input-25-78008352a689>:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  print(model.similarity('man', 'king'))
<ipython-input-25-78008352a689>:2: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  print(model.similarity('woman', 'king'))


In [30]:
# or to use the spicy version:
word_1 = model.wv['man']
word_2 = model.wv['king']
print(1 - cosine(word_1, word_2))

word_1 = model.wv['woman']
word_2 = model.wv['king']
print(1 - cosine(word_1, word_2))

0.38541901111602783
0.22175857424736023


# Part 2: Reproducing Wevers

1. For his analysis, Wevers relies on using lists of words within certain categories (money-related words/job-related words, etc). You can find English versions of the lists he uses in word cats.p. Load this data frame and inspect the list of occupations. Unlike English, compounds in Dutch are (normally) not written separately. Thus, in Dutch credit card is written as creditcard (or: kredietkaart). Why would that pose a possible problem if we want to reproduce Wevers’s analysis on English texts?

In [32]:
word_cats = pickle.load(open('/Users/jaspervogelzang/Documents/ADS Master/Data Mining/Lab Sessions/word_cats.p', 'rb'))

In [60]:
word_cats

,affect,posemo,negemo,social,family,cogproc,percept,body,work,leisure,money,relig,occupation
0,protesting,incentive,destruction,chick,ma's,comply,squeez,pussy,dotcom,dnd,portfolio,goddess,accountant
1,pretty,luck,beaten,ma's,niece,luck,sand,wears,employee,vacation,sale,karma,actor
2,sighs,freeing,battl,lets,stepkid,unquestion,moist,hearts,paper,hobb,stores,pastor,actress
3,warmth,pretty,protesting,son's,son's,pretty,warmth,asleep,earns,band,bets,temple,actuary
4,mooch,nicely,dumber,daddies,daddies,become,gloomy,gums,assign,skat,bank,holy,acupuncturist
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,ty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1391,undesir,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1392,trembl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1393,weaken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. Make a list of male-related words (he, his, male, etc.) and of female- related words. Get the mean embeddings of those words.

In [84]:
male = ['male', 'men','boy', 'guy', 'his', 'him', 'dude']
female = ['female', 'her', 'feminine', 'women', 'lady', 'sister', 'granny', 'daughter']

words = [word for word in male if word in model.wv.vocab] # checks if word is in vocabulary (i.e. has been seen by the model before)
mean_embedding_male = np.mean([model.wv[words] for words in male], axis=0)
print('Mean embedding for male words: ', mean_embedding_male.shape)

words = [word for word in female if word in model.wv.vocab] # checks if word is in vocabulary (i.e. has been seen by the model before)
mean_embedding_female = np.mean([model.wv[words] for words in female], axis=0)
print('Mean embedding for female words: ', mean_embedding_female.shape)

Mean embedding for male words:  (300,)
Mean embedding for female words:  (300,)


['female', 'her', 'feminine', 'women', 'lady', 'sister', 'granny', 'daughter']

3. To get an indication of the gender bias related to a certain category, Wevers uses the following method:
First, he loops over all the words within a certain category (e.g. occupa- tions). For each word in the category, he calculates (1) the (Frobenius- based) distance between that word and the mean embedding of the male related words, and (2) the distance between that word and the mean embedding of the female related words.
Then, (3), he calculates, for each word in the category, the difference between (1) and (2) by subtracting them, This could be seen as the gender bias of a specific word.
Finally, to get the gender bias within a certain category, he takes the mean of the gender bias for each word in the category.
Try to implement this approach using the words in the column occupation. When you loop over the words within a category, also store the word and the bias of that words in a dictionary, and use that to make a data frame (also see the section Most distinctive words on how to make a data frame).
What occupations are most biased towards men, and what occupations are most biased towards women? What do the results say about the gender bias in these journals? And what do they say about the word embeddings method in general?